In [6]:

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama
import gradio as gr
import time
import os.path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    SummaryIndex,
    load_index_from_storage,
)
import shutil
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector
from llama_index.core.node_parser import SentenceSplitter

## Test Llama-index with local Ollama models (llama3 and mxbai-embed)

In [ ]:
documents = SimpleDirectoryReader("data").load_data()

# bge-base embedding model
Settings.embed_model = OllamaEmbedding(model_name="mxbai-embed-large:latest")

# ollama
Settings.llm = Ollama(model="llama3:latest", request_timeout=360.0)

index = VectorStoreIndex.from_documents(
    documents,
)
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")
print(response)

The author grew up working on writing and programming. Before college, he wrote short stories and tried to write programs on an IBM 1401 using an early version of Fortran. Later, with the advent of microcomputers, he started programming in earnest and built his own computer, a TRS-80, which was a kit sold by Heathkit.


## Testing out ReAct Agent 

In [5]:
from llama_index.core.tools import FunctionTool, ToolMetadata
from llama_index.llms.ollama import Ollama
from llama_index.core.agent import ReActAgent

# define sample Tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

# initialize llm
llm = Ollama(model="llama3:latest")

# initialize ReAct agent
agent = ReActAgent.from_tools([multiply_tool], llm=llm, verbose=True)

response = agent.chat("What is 2 *617?")
print(response.response)

> Running step def9f7a6-839f-40a7-8dc6-9f28a418f708. Step input: What is 2 *617?
Thought: The current language of the user is: not specified. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {'a': 2, 'b': 617}
Observation: 1234
> Running step fb5abbaf-4c46-42e9-93c1-920029f5ea7e. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: 1234 is not the result of the multiplication, but rather a separate number. To get the correct answer, we can continue using the multiply tool.
1234 is not the result of the multiplication, but rather a separate number. To get the correct answer, we can continue using the multiply tool.


## Chatbot (main functions)

In [18]:
# check if storage already exists
PERSIST_DIR = "./storage"
# if not os.path.exists(PERSIST_DIR):
#     # load the documents and create the index
#     documents = SimpleDirectoryReader("data").load_data()
#     index = VectorStoreIndex.from_documents(documents)
#     # store it for later
#     index.storage_context.persist(persist_dir=PERSIST_DIR)
# else:
#     # load the existing index
#     storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
#     index = load_index_from_storage(storage_context)


def print_like_dislike(x: gr.LikeData):
    print(x.index, x.value, x.liked)


def add_message(history, message):
    for x in message["files"]:
        history.append({"role": "user", "content": {"path": x}})
        ## Code to load each file into vector index
        source = x
        filename = os.path.basename(x)
        destination = './data/'+filename
        try:
            # Copy the file from `source` to `destination`
            shutil.copy(source, destination)
            print(f"File copied successfully from {source} to {destination}")
        except FileNotFoundError as e:
            print(f"The specified source file does not exist: {e}")
        except PermissionError as e:
            print(f"Permission denied while accessing the files or directories: {e}")
        except IsADirectoryError as e:
            # This exception occurs if `destination` is a directory instead of a file
            print(f"Destination should be a file, but it's a directory: {e}")
        except Exception as e:
            print(f"An error occurred while copying the file: {e}")

    if message["text"] is not None:
        history.append({"role": "user", "content": message["text"]})
    return history, gr.MultimodalTextbox(value=None, interactive=False)


def bot(history: list):
    # response = "**That's cool!**"
    documents = SimpleDirectoryReader("data").load_data()
    # splitter = SentenceSplitter(chunk_size=1024)
    # nodes = splitter.get_nodes_from_documents(documents=documents)
    
    # bge-base embedding model
    Settings.embed_model = OllamaEmbedding(model_name="mxbai-embed-large:latest")

    # ollama
    Settings.llm = Ollama(model="qwen2.5:72b-instruct-q2_K", request_timeout=360.0)

    vector_index = VectorStoreIndex.from_documents(documents=documents)
    summary_index = SummaryIndex.from_documents(documents=documents)
    summary_query_engine = summary_index.as_query_engine()
    vector_query_engine = vector_index.as_query_engine(similarity_top_k=3)
    
    summary_tool = QueryEngineTool.from_defaults(query_engine=summary_query_engine, 
                                                 description="Useful for summarization questions related to given document")
    vector_tool = QueryEngineTool.from_defaults(query_engine=vector_query_engine,
                                                description=("Useful for retrieving specific context from the given document"))
    query_engine = RouterQueryEngine(
        selector=LLMSingleSelector.from_defaults(),
        query_engine_tools=[
            vector_tool, 
            # summary_tool
        ], 
        verbose=True 
        
    )

    # query_engine_tools = [QueryEngineTool(query_engine=query_engine, 
    #                                       metadata=ToolMetadata(
    #                                           name="Uploaded file", 
    #                                           description="Uploaded file information"),
    #                                         )
        
    # ]
    
    # # initialize llm
    # llm = Ollama(model="llama3:latest")

    # # initialize ReAct agent
    # agent = ReActAgent.from_tools([query_engine_tools], llm=llm, verbose=True)
    # # print(history[-1]['content'])

    
    response = query_engine.query(history[-1]['content'])
    # response = agent.chat(history[-1]['content'])
    # print(response)
    # print(type(response))
    # print(response.response)
    history.append({"role": "assistant", "content": ""})
    for character in response.response:
        history[-1]["content"] += character
        time.sleep(0.05)
        yield history
    # return history


## Chatbot UI

In [19]:

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(elem_id="chatbot", bubble_full_width=False, type="messages")

    chat_input = gr.MultimodalTextbox(
        interactive=True,
        file_count="multiple",
        placeholder="Enter message or upload file...",
        show_label=False,
    )

    chat_msg = chat_input.submit(
        add_message, [chatbot, chat_input], [chatbot, chat_input]
    )
    bot_msg = chat_msg.then(bot, chatbot, chatbot, api_name="bot_response")
    bot_msg.then(lambda: gr.MultimodalTextbox(interactive=True), None, [chat_input])

    chatbot.like(print_like_dislike, None, None, like_user_message=True)

demo.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


File copied successfully from /tmp/gradio/d1d658be95b28f084190e9113783c54974efc0fa669e901ad3fd48b68a9c3afc/2309.02427v3.pdf to ./data/2309.02427v3.pdf
Selecting query engine 0: The summary suggests that the content of the document can be retrieved, which implies that it contains information about the title as well..
Selecting query engine 0: The question 'What is the Cognitive Architecture for language agents?' requires specific information that can be retrieved from a relevant document, making it the most suitable use case for the provided option..


## Close Demo and delete files

In [21]:
demo.close()
# Specify the directory path
directory_path = './data'

def delete_files_recursively(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Construct the full file path
            file_path = os.path.join(root, file)
            try:
                # Delete the file
                os.remove(file_path)
                print(f"Deleted: {file_path}")
            except Exception as e:
                print(f"Error deleting {file_path}: {e}")

# Call the function to delete files recursively
delete_files_recursively(directory_path)

Closing server running on port: 7861
